In [1]:
from datasets import load_dataset
import tqdm
import pandas as pd
from transformers import pipeline

/opt/homebrew/Caskroom/mambaforge/base/envs/krakowiak/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [185]:
USER_TAG = '<|Użytkownik|>: '
ASSISTANT_TAG = '<|Asystent-AI|>: '

### QA - Process Klej_dyk

In [186]:
klej_dyk = load_dataset("allegro/klej-dyk")
klej_dyk_df_train = klej_dyk['train'].to_pandas()
klej_dyk_df_test = klej_dyk['test'].to_pandas()
# combine train and test
klej_dyk_df = pd.concat([klej_dyk_df_train, klej_dyk_df_test], ignore_index=True)
klej_dyk_df = klej_dyk_df[klej_dyk_df["target"]==1][["question","answer"]]
klej_dyk_df["question"] = klej_dyk_df["question"].str.capitalize()
#### CREATE PROMPTS
klej_dyk_df["question"] = USER_TAG + klej_dyk_df["question"]
klej_dyk_df["answer"] =  ASSISTANT_TAG + klej_dyk_df["answer"]
#### PROMPTS
prompts_klej_dyk = klej_dyk_df["question"] + klej_dyk_df["answer"]
prompts_klej_dyk

2       <|Użytkownik|>: Jakie choroby genetyczne dzied...
5       <|Użytkownik|>: Czym zajmowali się świątnicy?<...
6       <|Użytkownik|>: Kto w księdze metrykalnej opac...
17      <|Użytkownik|>: Kto był tłumaczem języka angie...
19      <|Użytkownik|>: Jak promieniowanie słoneczne m...
                              ...                        
5166    <|Użytkownik|>: Jaką pomoc można otrzymać pod ...
5169    <|Użytkownik|>: Dlaczego w chorobie hartnupów ...
5174    <|Użytkownik|>: Z jakimi zbrodniarzami nazisto...
5178    <|Użytkownik|>: Skąd wzięła swą nazwę jedna z ...
5180    <|Użytkownik|>: Gdzie ma początek magistrala t...
Length: 875, dtype: object

### GPT-EXAM

In [184]:
gpt_exam = load_dataset("sdadas/gpt-exams")
gpt_exam_df = gpt_exam['train'].to_pandas()
#### CREATE PROMPTS
gpt_exam_df["question"] = USER_TAG + gpt_exam_df["question"]
gpt_exam_df["answer"] = ASSISTANT_TAG + gpt_exam_df["answer"]
#### PROMPTS
prompts_gpt_exam = gpt_exam_df["question"] + gpt_exam_df["answer"]
prompts_gpt_exam


0       <|użytkownik|> Kim był Donatello i jakie są je...
1       <|użytkownik|> Jakie są główne cechy systemów ...
2       <|użytkownik|> Jakie są najważniejsze metody z...
3       <|użytkownik|> Jakie są możliwości leczenia ja...
4       <|użytkownik|> Jakie są przyczyny i leczenie z...
                              ...                        
8126    <|użytkownik|> Co to jest amortyzacja? Jakie s...
8127    <|użytkownik|> Jaka jest rola lekarza medycyny...
8128    <|użytkownik|> Jakie są prawa człowieka dotycz...
8129    <|użytkownik|> Jakie są podstawowe instrumenty...
8130    <|użytkownik|> Jakie są różnice między konfigu...
Length: 8131, dtype: object

### Guanco

In [5]:
oa_guanco = load_dataset("timdettmers/openassistant-guanaco")
oa_guanco_df = oa_guanco['train'].to_pandas()

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", truncation=True, max_length=128)
oa_guanco_en = []
for i,text in enumerate(tqdm.tqdm(oa_guanco_df["text"])):
    if pipe(text)[0]["label"] == "en":
        oa_guanco_en.append(text)

### temp df
df = pd.DataFrame(oa_guanco_en, columns=["text"])
df = df[~df["text"].str.contains("Open Assistant")]
df = df[~df["text"].str.contains("grammar")]
df = df[~df["text"].str.contains("Re-write this sentence")]
df = df[~df["text"].str.contains("Rewrite")]

## replace ### Asystent with <ASYSTENT_TAG>
df["text"] = df["text"].str.replace("### Assistant", "<ASYSTENT_TAG>")
## replace ### Użytkownik with <USER_TAG>
df["text"] = df["text"].str.replace("### Human", "<USER_TAG>")

df.to_csv("../data/en/oa_guanco_en.csv", index=False)

  0%|          | 33/9846 [00:02<13:21, 12.24it/s]


KeyboardInterrupt: 

In [8]:
## replace ### Asystent with <ASYSTENT_TAG>
df["text"] = df["text"].str.replace("### Assistant", "<ASYSTENT_TAG>")
## replace ### Użytkownik with <USER_TAG>
df["text"] = df["text"].str.replace("### Human", "<USER_TAG>")
df.to_csv("../data/en/oa_guanco_en.csv", index=False)

NameError: name 'df' is not defined

In [9]:
oa_translated_df = pd.read_csv("oa_dataset_translated.csv")
oa_translated_df = oa_translated_df["translation"]
## create new column in new df named oa_prompts with translation
oa_translated_df = oa_translated_df.str.replace("<ASYSTENT_TAG>", "### Asystent")
prompts_oa = oa_translated_df.str.replace("<USER_TAG>", "### Użytkownik")

FileNotFoundError: [Errno 2] No such file or directory: 'oa_dataset_translated.csv'

#### Dolly 15K

In [10]:
dolly = load_dataset("databricks/databricks-dolly-15k")
dolly_train_df = dolly['train'].to_pandas()
# to csv
dolly_train_df.to_csv("../data/en/dolly_train.csv", index=False)

OSError: Cannot save file into a non-existent directory: '../data/en'

### QA - with context - Process clarin-pl/poquad

In [11]:
poquad = load_dataset("clarin-pl/poquad")
poquad_df_train = poquad['train'].to_pandas()
poquad_df_test = poquad['validation'].to_pandas()
poquad_df = pd.concat([poquad_df_train, poquad_df_test], ignore_index=True)
poquad_df = poquad_df.drop_duplicates(subset=["title"]).reset_index(drop=True)
poquad_df["answers"] = poquad_df["answers"].apply(lambda x: x["text"][0])
poquad_df["answers"] = poquad_df["answers"].str.capitalize()
poquad_df = poquad_df[["context","question","answers"]]
poquad_df
#### CREATE PROMPTS
poquad_df["question_uzytownik"] = USER_TAG + poquad_df["question"]
poquad_df["answer_asystent"] = ASSISTANT_TAG + poquad_df["answers"]


'qodpowiedz korzystając z tekstua'

### Process poleval_2019_mt

In [12]:
opus100 = load_dataset("opus100", "en-pl")

In [13]:
opus100_df = opus100['validation'].to_pandas()
#get data from dictonary inside column translation
opus100_df["en"] = opus100_df["translation"].apply(lambda x: x["en"])
opus100_df["pl"] = opus100_df["translation"].apply(lambda x: x["pl"])
## przetłumacz teksty z angielskiego na polski z pierwszego 1000 wierszy
# Take first 1000 rows
opus100_df_en_pl = opus100_df[:1000]
opus100_df_en_pl["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski. " + opus100_df_en_pl["en"] + "### Asystent: " + opus100_df_en_pl["pl"]

opus100_df_pl_en = opus100_df[1000:2000]
opus100_df_pl_en["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Polskiego na Angielski. " + opus100_df["pl"] + "### Asystent: " + opus100_df["en"]

#### concat 
prompts_opus100 = pd.concat([opus100_df_en_pl["prompt"], opus100_df_pl_en["prompt"]], ignore_index=True)
# reset index
prompts_opus100 = prompts_opus100.reset_index(drop=True)


/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_87122/2643363535.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opus100_df_en_pl["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski. " + opus100_df_en_pl["en"] + "### Asystent: " + opus100_df_en_pl["pl"]
/var/folders/fn/clqd998j3vn03yyqqybzkq240000gn/T/ipykernel_87122/2643363535.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opus100_df_pl_en["prompt"] = "### Użytkownik: Przetłumacz dany tekst z języka Pols

### JOIN dolly prompts

In [112]:
synonimy_do_tekst_źródłowy = [
    "Materiał źródłowy",
    "Tekst bazowy",
    "Dokument źródłowy",
    "Materiał badawczy",
    "Zbiór danych",
    "Plik źródłowy",
    "Zawartość źródłowa",
    "Tekst oryginalny",
    "Zasoby źródłowe",
    "Dane do analizy",
    "Tekst referencyjny",
    "Materiał wejściowy",
    "Dane pierwotne",
    "Materiał analizy",
    "Informacje źródłowe",
    "Tekst podstawowy",
    "Dokument do analizy",
    "Tekst do badania",
    "Tekst do oceny",
    "Tekst",
    "Tekst bazowy do analizy",
    "Materiał referencyjny",
    "Dane badawcze",
    "Materiał do przetwarzania",
    "Treść źródłowa",
    "Informacje do badania",
    "Tekst poddawany analizie",
    "Dane eksploracyjne",
    "Tekst wejściowy",
    "Materiał do badania",
    "Dane źródłowe",
    "Tekst do kontroli",
    "Zbiór tekstowy",
    "Materiał interpretacyjny",
    "Tekst do udzielenia odpowiedzi",
    "Dane do oceny",
    "Plik badawczy",
    "Materiał referencyjny",
    "Plik do analizy",
    "Zawartość do analizy",
    "Tekst do przetwarzania",
    "Dokument referencyjny",
    "Tekst do interpretacji",
    "Materiał do oceny",
    "Informacje do przetwarzania",
    "Plik referencyjny",
    "Tekst do badań",
    "Materiał eksploracyjny"
]


In [199]:
### Load translated dataset
dolly_translated_df = pd.read_csv("../../data/pl_eval/dolly_pl.csv")
dolly_translated_df = dolly_translated_df[dolly_translated_df["trans_score_pl"] >= 0.8]
dolly_translated_df["text"] = ''
# join prompts
for i, row in dolly_translated_df.iterrows():
    dolly_translated_df.at[i,"text"] = USER_TAG + row["instruction_pl"] + ' '+row["context_pl"] + ASSISTANT_TAG + row["response_pl"]
    # dolly_translated_df.at[i,"text"] = USER_TAG + row["instruction_pl"] + f"\n{synonimy_do_tekst_źródłowy[i%len(synonimy_do_tekst_źródłowy)]}\n" + row["context_pl"] + ASSISTANT_TAG + row["response_pl"]
# rename column to text
dolly_translated_df.reset_index(drop=True, inplace=True)
dolly_translated_df = dolly_translated_df[["text"]]
dolly_translated_df

,text
0,<|Użytkownik|>: Kiedy Virgin Australia rozpocz...
1,<|Użytkownik|>: Kiedy urodziła się Tomoaki Kom...
2,<|Użytkownik|>: Biorąc pod uwagę tekst referen...
3,<|Użytkownik|>: Czym jest wydobycie procesowe?...
4,<|Użytkownik|>: Co to jest pryzmat dyspersyjny...
...,...
2196,<|Użytkownik|>: Kto wyreżyserował Terror Mount...
2197,<|Użytkownik|>: Ile typów kotów rozpoznaje świ...
2198,"<|Użytkownik|>: Kiedy Phil Knight ogłosił, że ..."
2199,<|Użytkownik|>: Jakie kwiaty można znaleźć w P...


In [214]:
dolly_translated_df["text"].iloc[random.randint(0, 2201)]

'<|Użytkownik|>: W którym dystrykcie jest Lightwater? Lightwater – wieś w Anglii, w dystrykcie Surrey Heath, w hrabstwie Surrey, około 43\xa0km na południowy zachód od centrum Londynu. Bezpośrednio otaczające miasta i wsie to Bagshot, Deepcut, Windlesham, Camberley i West End, Woking. Na północny-zachód jest ograniczony autostradą M3, a na północny-wschód przez “Lightwater Bypass” (A322). Na południowym wschodzie jest ograniczony przez “Red Road” (B311), a na południu przez istnieją tory testowe pojazdów należące do Ministerstwa Obrony.\n\nMiejscowość przylega do zjazdu nr 3 z autostrady M3. Najbliższa stacja kolejowa znajduje się w Bagshot, na linii Ascot i Aldershot. Lightwater jest częścią cywilnej parafii Windlesham, która liczy 17,000 mieszkańców i obejmuje również sąsiednią wioskę Bagshot.<|Asystent|>: Lightwater – miasto w hrabstwie Windlesham, w dystrykcie Surrey Heath.'

#### JOIN GUANCO PROMPTS

In [154]:
def remove_duplicate_words_and_following(input_string):
    words = input_string.split()  # Split the string into words
    unique_words = []
    seen_words = set()

    for word in words:
        if word not in seen_words:
            seen_words.add(word)
            unique_words.append(word)

    return ' '.join(unique_words)

input_string = "This is an example example of removing duplicate words and words that come after."
result = remove_duplicate_words_and_following(input_string)
print(result)

This is an example of removing duplicate words and that come after.


In [159]:
text

'<|użytkownik|> Gdzie jest Brian?<|asystent|> Istnieje wiele osób o imieniu Brian. Czy możesz podać więcej informacji, takich jak nazwisko Briana lub inne informacje identyfikujące?<|użytkownik|> Gra na gitarze, kręcone włosy, nie pochodzi z USA<|asystent|> Być może odnosisz się do Briana Harolda Maya, angielskiego muzyka znanego na całym świecie z bycia gitarzystą zespołu rockowego Queen. Ma też kręcone włosy, więc pasuje do twojego opisu. Jeśli chodzi o miejsce jego pobytu, wydaje się, że obecnie mieszka w południowo-wschodnim Londynie, choć te informacje mogą być niedokładne.'

['<|użytkownik|> Gdzie jest Brian?<|asystent|> Istnieje wiele osób o imieniu Brian. Czy możesz podać więcej informacji, takich jak nazwisko Briana lub inne informacje identyfikujące?<|użytkownik|> Gra na gitarze, kręcone włosy, nie pochodzi z USA<|asystent|> Być może odnosisz się do Briana Harolda Maya, angielskiego muzyka znanego na całym świecie z bycia gitarzystą zespołu rockowego Queen. Ma też kręcone włosy, więc pasuje do twojego opisu. Jeśli chodzi o miejsce jego pobytu, wydaje się, że obecnie mieszka w południowo-wschodnim Londynie, choć te informacje mogą być niedokładne.']

In [108]:
guanco_translated_df = pd.read_csv("../../data/pl_eval/guanco_pl.csv")
guanco_translated_df["text_pl"] = guanco_translated_df["text_pl"].str.replace("<ASYSTENT_TAG>: ", ASSISTANT_TAG)
guanco_translated_df["text"] = guanco_translated_df["text_pl"].str.replace("<USER_TAG>: ", USER_TAG)
guanco_translated_df["text"] = guanco_translated_df["text"].str.replace("Open Assistant", "Krakowiak")
guanco_translated_df = guanco_translated_df[guanco_translated_df["trans_score_pl"] >= 0.8]["text"]
guanco_translated_df.reset_index(drop=True, inplace=True)
guanco_translated_df
guanco_translated_df = pd.DataFrame(guanco_translated_df, columns=["text"])

In [109]:
guanco_translated_df= guanco_translated_df['text'].apply(split_after_second_occurrence)


In [111]:
guanco_translated_df.iloc[1]

'<|użytkownik|> Gdzie jest Brian?<|asystent|> Istnieje wiele osób o imieniu Brian. Czy możesz podać więcej informacji, takich jak nazwisko Briana lub inne informacje identyfikujące?<|użytkownik|> Gra na gitarze, kręcone włosy, nie pochodzi z USA<|asystent|> Być może odnosisz się do Briana Harolda Maya, angielskiego muzyka znanego na całym świecie z bycia gitarzystą zespołu rockowego Queen. Ma też kręcone włosy, więc pasuje do twojego opisu. Jeśli chodzi o miejsce jego pobytu, wydaje się, że obecnie mieszka w południowo-wschodnim Londynie, choć te informacje mogą być niedokładne.'

In [183]:
def remove_after_second_occurrence_from_right(input_string, substring):
    parts = input_string.rsplit(substring, 2)
    if len(parts) == 3:
        result_string = parts[0] + substring + parts[1]
        return result_string
    else:
        return input_string

input_string = guanco_translated_df.iloc[5]
substring = USER_TAG

result = remove_after_second_occurrence_from_right(input_string, substring)
print(result)


<|użytkownik|> kilka interesujących faktów o astronomii.<|asystent|> Oto kilka interesujących faktów o astronomii:

1. Galaktyka Drogi Mlecznej – która jest domem dla Ziemi i reszty Układu Słonecznego – szacuje się, że zawiera około 100 miliardów gwiazd.
2. Uważa się, że Wszechświat rozszerza się w przyspieszającym tempie i uważa się, że przyspieszenie to jest spowodowane ciemną energią.
3. Układ Słoneczny znajduje się w jednym z ramion spiralnych Drogi Mlecznej, znanym jako ramię Oriona lub ramię Cariny.
4. Księżyc jest jedynym naturalnym satelitą Ziemi i krąży wokół Ziemi od ponad 4.5 miliarda lat.
5. Saturn jest jedyną planetą w Układzie Słonecznym, która ma system pierścieni, a pierścienie planety składają się głównie z cząstek lodu wodnego.
6. Merkury jest planetą najbliższą Słońcu i okrąża Słońce w zaledwie 88 dni.
7. Mars jest nieco mniejszy od Ziemi i znany jest z unikalnego czerwonego koloru, który jest spowodowany obecnością tlenku żelaza na powierzchni planety.


In [27]:
os_wizard = pd.read_csv("../../data/pl_eval/oa_wizard_pl.csv")
os_wizard["text_pl"] = os_wizard["text_pl"].str.replace("<ASYSTENT_TAG>: ", ASSISTANT_TAG)
os_wizard["text"] = os_wizard["text_pl"].str.replace("<USER_TAG>: ", USER_TAG)
# trans_score_pl >= 0.8 get those rows
os_wizard_translated_df = os_wizard[os_wizard["trans_score_pl"] >= 0.8]["text"]
os_wizard_translated_df.reset_index(drop=True, inplace=True)
os_wizard_translated_df

0       <|użytkownik|> W którym roku wynaleziono pierw...
1       <|użytkownik|> Zaproponuj pytanie do zadania p...
2       <|użytkownik|> Jaka jest formuła konstruowania...
3       <|użytkownik|> Kto jest autorem powieści “Biał...
4       <|użytkownik|> Przypisz ten zawód do odpowiedn...
                              ...                        
7086    <|użytkownik|> Jakiego języka powinienem się n...
7087    <|użytkownik|> witaj\r\n<|asystent|> Witaj, ja...
7088    <|użytkownik|> Alicja podróżuje z dala od punk...
7089    <|użytkownik|> jaka jest dzisiaj pogoda?\r\n<|...
7090    <|użytkownik|> Podaj mi dwie liczby, tak aby i...
Name: text, Length: 7091, dtype: object

In [17]:
prompts_klej_dyk.iloc[0]

'<|użytkownik|> Jakie choroby genetyczne dziedziczą się autosomalnie dominująco?<|asystent|> Nerwiakowłókniakowatości (neurofibromatozy, łac. neurofibromatoses) są grupą chorób genetycznych dziedziczonych autosomalnie dominująco cechujących się obecnością licznych, niezłośliwych nowotworów pochodzenia nerwowego.'

In [18]:
prompts_gpt_exam.iloc[0]

'<|użytkownik|> Kim był Donatello i jakie są jego najważniejsze dzieła?<|asystent|> Donatello, właśc. Donato di Niccolò di Betto Bardi (1386/1387 - 13 grudnia 1466), był włoskim rzeźbiarzem i złotnikiem zaangażowanym w ruch renesansowy. Uważany jest za jednego z najważniejszych rzeźbiarzy w historii, który przyczynił się do rewolucji w sztuce i odnowy rzemiosła rzeźbiarskiego.\nJego prace są często opisane jako przełomowe i innowacyjne, zwracające uwagę na nowatorskie podejście do kompozycji, formy i techniki. Wprowadził trójwymiarowość, naturalizm i emocjonalność do rzeźby, przyczyniając się do rozwoju perspektywy rzeźby.\nNajważniejsze i najbardziej znane dzieła Donatella to:\n1. David: Ta rzeźba z brązu przedstawia postać biblijnego Davida. Jest to jedno z najbardziej wpływowych dzieł Donatella, które odzwierciedla jego umiejętność ukazania dynamizmu i naturalizmu postaci.\n2. Święty Jerzy: Rzeźba wykonana dla Florenckiego Konsulatu we Florencji przedstawia świętego Jerzego walczące

In [32]:
prompts_opus100.iloc[0]

'### Użytkownik: Przetłumacz dany tekst z języka Angielskiego na Polski. Jab it into his antecubital vein, and tape his arm down.### Asystent: Wbij w żyłę w zgięciu łokciowym i opatrz rękę.'

In [31]:
os_wizard_translated_df.iloc[0]

'<|użytkownik|> W którym roku wynaleziono pierwszą mysz komputerową?\r\n<|asystent|> Pierwsza mysz komputerowa została wynaleziona w 1964 roku.\r\n<|użytkownik|> Kto wynalazł pierwszą mysz komputerową?\r\n<|asystent|> Pierwsza mysz komputerowa została wynaleziona przez Douglasa Engelbarta w 1964 roku podczas pracy w Instytucie Badawczym Stanforda.'

In [21]:
dolly_translated_df.iloc[0]

'<|użytkownik|> Kiedy Virgin Australia rozpoczęła działalność?\ntekst źródłowy:\nVirgin Australia, nazwa handlowa Virgin Australia Airlines Pty Ltd, jest australijską linią lotniczą. Jest to największa linia lotnicza pod względem wielkości floty, która używa marki Virgin. Rozpoczęła działalność 31 sierpnia 2000 roku pod nazwą Virgin Blue, z dwoma samolotami na jednej trasie. Nagle stała się główną linią lotniczą na australijskim rynku krajowym po upadku Ansett Australia we wrześniu 2001 roku. Od tego czasu linia rozrosła się do 32 miast w Australii, z węzłów w Brisbane, Melbourne i Sydney.<|asystent|> Virgin Australia rozpoczął służbę 31 sierpnia 2000 jako Virgin Blue, z dwoma samolotami na jednej trasie.'

In [33]:
## concat all prompts
prompts = pd.concat([prompts_klej_dyk, prompts_gpt_exam, os_wizard_translated_df, guanco_translated_df, dolly_translated_df], ignore_index=True)
## shuffle rows in prompts 
prompts = prompts.sample(frac=1).reset_index(drop=True)
## add column NAMED text
prompts = pd.DataFrame(prompts, columns=["text"])
prompts

,text
0,<|użytkownik|> Jakie zastosowanie mają włókna ...
1,<|użytkownik|> Jakie procesy zachodzą w cyklu ...
2,<|użytkownik|> Klasyfikuj następujące zdania n...
3,<|użytkownik|> Jaki byłby odpowiedni nagłówek ...
4,<|użytkownik|> Jakie są podstawowe kategorie a...
...,...
18979,<|użytkownik|> Jakie są najważniejsze cechy ch...
18980,<|użytkownik|> Jakie są główne różnice między ...
18981,<|użytkownik|> Jakie są główne zastosowania ko...
18982,<|użytkownik|> Jakie są najważniejsze cechy be...


In [34]:
prompts_gpt_exam

0       <|użytkownik|> Kim był Donatello i jakie są je...
1       <|użytkownik|> Jakie są główne cechy systemów ...
2       <|użytkownik|> Jakie są najważniejsze metody z...
3       <|użytkownik|> Jakie są możliwości leczenia ja...
4       <|użytkownik|> Jakie są przyczyny i leczenie z...
                              ...                        
8126    <|użytkownik|> Co to jest amortyzacja? Jakie s...
8127    <|użytkownik|> Jaka jest rola lekarza medycyny...
8128    <|użytkownik|> Jakie są prawa człowieka dotycz...
8129    <|użytkownik|> Jakie są podstawowe instrumenty...
8130    <|użytkownik|> Jakie są różnice między konfigu...
Length: 8131, dtype: object

In [125]:
import random

In [63]:
prompts["text"].iloc[random.randint(0, len(prompts))]

'<|użytkownik|> Na jakiej zasadzie działają hormonalne środki antykoncepcyjne?<|asystent|> Hormonalne środki antykoncepcyjne, znane również jako tabletki antykoncepcyjne, działają na zasadzie wpływu na naturalny cykl hormonalny organizmu kobiety. Składają się one z dwóch głównych hormonów: estrogenów i progestagenów.\nEstrogeny, takie jak etynyloestradiol, często używane są w tabletkach antykoncepcyjnych, ponieważ hamują wydzielanie hormonalne związków gonadotropowych, czyli hormonów odpowiedzialnych za regulację cyklu menstruacyjnego. Estrogeny blokują wydzielanie hormonu folikulotropowego (FSH) z przysadki mózgowej, co powoduje zahamowanie dojrzewania pęcherzyków jajnikowych i uniemożliwienie owulacji.\nProgestageny, takie jak lewonorgestrel, drospirenon lub desogestrel, działają głównie poprzez hamowanie wydzielania hormonu luteinizującego (LH) przez przysadkę mózgową. Chociaż progestageny mają różne mechanizmy działania, ogólnie mają podobny efekt na jajniki jak estrogeny, uniemożl

In [ ]:
### Customr prompts
"### Uzytkownik: Kim jesteś? Jestem"

### Post to huggingface dataset

In [64]:
from datasets import load_dataset
import datasets
from datasets import Dataset, DatasetDict

### get random 1000 rows from prompts with seed and put into validation df
prompts_sample = prompts.sample(n=500, random_state=1)
prompts_sample = prompts_sample.reset_index(drop=True)
### substract validation df from prompts df
prompts_train = prompts[~prompts.text.isin(prompts_sample.text)]
prompts_train = prompts_train.reset_index(drop=True)
### reset index

In [65]:
ds = DatasetDict()
ds['train'] = Dataset.from_pandas(prompts_train)
ds['validation'] = Dataset.from_pandas(prompts_sample)

In [66]:
ds.push_to_hub("szymonrucinski/krakowiak-pl")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.45s/it]
